![HPEDEVlogo](Pictures/hpedevlogo-NB.JPG)    ![Dockerlogo](Pictures/docker.png)  

Let's start by connecting to our docker appliance again (Docker in Docker Appliance)

In [ ]:
%login {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}

# Configuring nextcloud in a container

Estimated time: 60 minutes.

Based on the work done in the Docker Dojo during a Grenoble Docker Meetup (cf: https://github.com/Enalean/docker-dojo/tree/master/owncloud).

Nextcloud is a web based application providing services such as calendar data or file sharing e.g.
When we want to contain an application such as nextcloud, there are a certain number of aspects to take in account and solve:
  1. installing the application and its dependencies in the container
  2. allow IP configuration for remote access to the application
  3. allow data persistence at each invocation of the container
  4. allow configuration data persistence at each invocation of the container
  
One possibility would be to run the container from an image and launch the various commands in the container (as we've done previously). We could put that in a script and launch it systematically when we instantiate a container from an image, or rebuild a prepared image to be instantiated later. But there is a better way to achieve what we want to do, and this is by using the automation process by Docker with the Dockerfile.

The Dockerfile is a way to describe all the operations required to create an image from an initial basic one and stacking all the operations to build at the end the final image ready to be instantiated and consumed and thrown away.

Let's start our Dockerfile by creating a simple container from a base image and just installing some software components useful for our environment, and build an image from that:


In [ ]:
mkdir -p build
export HTTPPORT=`cat HTTPPORT`
echo 'FROM rockylinux/rockylinux:8' > build/Dockerfile
echo 'RUN dnf install -y httpd' >> build/Dockerfile
cat build/Dockerfile

In [ ]:
cd build && docker build . && cd ..

In [ ]:
docker image ls

So we can verify that a new Rocky Linux 8 image has been downloaded and based on it a new image has been created (without name nor tag, just an ID) containing httpd installed with its dependencies. 
Check it by instantiating a container based on that image and launching httpd in it:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId /usr/sbin/httpd

In [ ]:
ps auxww |  grep http

Strange, no httpd daemon seems to be running whereas you had no error message reported. Let's try to understand why. Look at the containers running:

In [ ]:
docker container ls

Ok, so you don't have any container running. Look at one which exited then:

In [ ]:
docker container ls -a

So here it is ! But as seen previously the container exited with a 0 status code, meaning no error. Why ? Well this is because the httpd process is giving back hand to the user immediately after launch, going into background. So Docker thinks that all the tasks that he had to do were done correctly and exit gracefully. Assess that analysis by issuing the following commands:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls -a | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId

So we checked that we can launch the httpd server from inside an instantiated container made from our image. It created the relevant log files in the container, with just a normal warning. We also checked how our image was built. Note that the image built is around 170 MB larger than the base Rocky Linux 8 one (shown by history) and has sensible modifications shown by the diff command.

It's a good start, but now we would like to have our httpd server started automatically with our container creation and staying alive after launch. And have attribution accordingly ;-) So let's make the following modifications to the Dockerfile

In [ ]:
echo 'MAINTAINER myself@mydomain.org' >> build/Dockerfile
echo 'CMD /usr/sbin/apachectl -DFOREGROUND -k start' >> build/Dockerfile
echo ""
cat build/Dockerfile

In [ ]:
cd build && docker build . && cd ..

You can remark that all the first steps are very quick. This is because Docker caches steps, and will not repeat them unless the Dockerfile changes. You can modify the Dockerfile by putting the `MAINTAINER` command as the second line and re-launch the build. You'll see that in that case Docker invalidates its cache and restarts.
Secondly, we modified the way our Apache Web server is started by forcing it to stay in foreground after launch. That way Docker will continue to keep alive the container.
Now start a container from that image to check the web server is indeed started

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
docker run -d $ImgId

Now check using the same previous commands that the container is indeed running correctly:

In [ ]:
docker container ls
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker diff $CtnId
echo ""
docker history $ImgId
echo ""
docker logs $CtnId
echo ""
ps auxww | grep httpd | grep -v docker

So you can see that now your httpd process is running, has produced more differences (a pid file has been created), more log content. It is also running with a UID (48) which is not known by the host, just the container. So now let's try to reach our newly launched webserver:

In [ ]:
curl http://localhost

Of course that doesn't work ;-) You would have been surprised otherwise don't you ? The major aspect here is that the httpd server you launched is run in an isolated environement, the container, that is NOT the host, so its localhost is different from the localhost of the host. Similarly its IP address is different from the one of the host. Proof:

In [ ]:
echo ""
echo "*** host ***"
echo ""
echo "IP: {{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}"
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId dnf install -y net-tools >& /dev/null
echo ""
echo -n "IP: "
docker exec $CtnId /usr/sbin/ifconfig | grep -A 2 eth0 | tail -2 | head -1 | awk '{print $2 }'

Convinced ? So now try *inside* the container to reach the Web server:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
docker exec $CtnId curl -s http://localhost

Success !! So you indeed have a running web server, inside a Rocky Linux 8 distribution container. But that's not fully what you want. What you want is to be able to reach it from outside of the container.

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** CTN $CtnId ***"
echo ""
CtnIP=`docker exec $CtnId ifconfig | grep -A 2 eth0 | grep inet | awk '{print $2}'`
echo ""
echo "*** using CTN IP $CtnIP ***"
echo ""
curl -s http://$CtnIP | grep Rocky
HostIP="{{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}"
echo ""
echo "*** using host IP $HostIP ***"
echo ""
curl -s http://$HostIP | grep Rocky


So it works when you use the container IP (because Linux and Docker set the routing up for you automagically, but it doesn't work if you use the IP address of the host, which is really what you want to provide access to for the external world. )

By default, the container ports are not exposed outside of the container. So you can't use your host OS to access your isolated webserver. 

You will have to explicitly redirect trafic arriving on port $HTTPPORT allocated to you on the host (which is accessible from outside your environment) to the container port 80 to allow access to the web server running in the container. This will require a change to the way you invoke the container creation. Let's stop the previous container and relaunch it with the right options:

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
docker run -d -p $HTTPPORT:80 $ImgId
echo ""
docker container ls
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
echo ""
echo "*** using host IP ***"
echo ""
HostIP={{ hostvars[inventory_hostname]['IP-WKSHP-Docker101'] }}
curl -s http://$HostIP:$HTTPPORT | grep Rocky

Now that we have redirected the port correctly, we can reach our webserver from outside the container using the host IP address. If that machine is reachable from the Internet, you have a Webserver in a container avaible online.

And as this is the case in our environment, please launch a Web browser of your choice and reach the URL mentioned in the next cell to check it, replacing the HTTPPORT variable by the value printed just in the next cell.

In [ ]:
echo Your HTTPPORT variable value is $HTTPPORT


Copy/paste that command in your browser launch bar: http://{{ JPHOSTEXT }}:$HTTPPORT

Don't you feel powerful now ?


It's now time to add some useful content to our web server !
Modify again the Dockerfile to add nextcloud to our image:

In [ ]:
ipaddr=`/sbin/ifconfig docker0 | head -2 | tail -1 | awk '{print $2}'`
echo "RUN curl ftp://$ipaddr/pub/nextcloud-22.1.1.zip -o /tmp/nextcloud.zip" >> build/Dockerfile
echo 'RUN dnf install -y unzip' >> build/Dockerfile
echo 'RUN cd /var/www/html/ && unzip -q /tmp/nextcloud.zip' >> build/Dockerfile
echo 'RUN chown apache:apache -R /var/www/html/nextcloud' >> build/Dockerfile
cat build/Dockerfile

Now re-create a new image based on that Dockerfile, purge the previous content before relaunching a new container. Note that we allow the apache user (running the httpd daemon to own the nextcloud data in order to avoid issues. the choice would be different in a production environement where security should be taken seriously).

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
echo ""
cd build && docker build . && cd ..
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
echo ""
docker run -d -p $HTTPPORT:80 $ImgId
docker container ls

Try now to connect to your nextcloud instance at  http://{{ JPHOSTEXT }}:$HTTPPORT/nextcloud. 

![Nextcloud failed](Pictures/nextcloud_without_dep.png)

1. What happened ?
2. What should you do next to solve the issue ?

Well, nexcloud required a PHP stack (>= 7.3)  to be working correctly. That's what the first line implies. So you probably need to add the nextcloud dependencies to be able to launch it. You can find details on how ton run nextcloud on Linux at their installation page : https://docs.nextcloud.com/server/latest/admin_manual/installation/source_installation.html 
We did the analysis job for you so you have now to modify your Dockerfile accordingly to solve that and relaunch the build/run of the image/container:

In [ ]:
echo "RUN dnf module enable -y php:7.4" >> build/Dockerfile
echo "RUN dnf install -y php php-dom php-mbstring php-pdo php-gd mariadb-server php-mysqlnd php-intl php-zip" >> build/Dockerfile
echo "RUN mkdir -p /run/php-fpm" >> build/Dockerfile
sed -i '/^CMD/d' build/Dockerfile
echo "CMD /usr/libexec/mysql-check-socket && /usr/libexec/mysql-prepare-db-dir && /usr/sbin/httpd && php-fpm && su mysql -s /usr/bin/bash -c '/usr/libexec/mysqld --basedir=/usr'" >> build/Dockerfile
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
echo ""
cd build && docker build . && cd ..
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
echo ""
docker run -d -p $HTTPPORT:80 $ImgId
docker container ls
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
sleep 5
docker exec $CtnId /usr/bin/mysqladmin -u root password 'linux1'

With that you should be able to use nextcloud !
Note that you need to use that version of nextcloud with Rocky Linux 8 for a PHP dependency management.
Note also that launching applications in a container (`CMD` line) doesn't benefit from the `systemd` facility and thus requires some analysis in order to launch them.

![Nextcloud login](Pictures/nextcloud_login.png)

But we're not done yet !!!

First log on to your nextcloud instance, and start customizing it: choose a login/passwd (admin/whatever), keep the data folder proposed by default (`/var/www/html/nextcloud/data`) and use SQLite for now. Please finish setup.

If you have a timeout on the gateway, please reload the page and log with the credential you just created. After the explanations, you should be able to see some files shared.

![Nextcloud dashboard](Pictures/nextcloud_dashboard.png)


As long as this very container is running, no problem to keep what you do. But what happens when you end the container ? 

But before, by now you have probably remarked that you have to each time deal with IDs for containers and images, which is not that convenient. Let's fix that by giving the `hpedevcloud` name to our image and container.

In [ ]:
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
CtnId=`docker container ls | grep $ImgId | awk '{print $1}'`
docker stop $CtnId
docker rm $CtnId
echo ""
cd build && docker build -t hpedevcloud . && cd ..
echo ""
ImgId=`docker image ls | head -2 | tail -1 | awk '{print $3}'`
echo ""
docker run -d -p $HTTPPORT:80 --name hpedevcloud hpedevcloud:latest
docker image ls
docker container ls

So now that we have `docker stop` and `docker rm` the container to relaunch it again, of course, none of the customization we made is kept as it's not part of your container content. Check that by reloading your browser again. You're back to customize it again. Annoying !

So we now have to deal with storage management for our Docker container. Meaning that we need to store all the nextcloud data outside of the container (here on our host) and attach them to the container at start so it's not evanescent when the container stops.

We've seen during the setup that we were asked the location of the data folder. So that seems like a good candidate to externalize. Let's work on that. Create a `data` directory on your host in your home directory, mount it in your container under `/data`, and then point your setup to it instead of the one under `/var/www/html/owncloud`.

In [ ]:
mkdir -p data
docker stop hpedevcloud
docker rm hpedevcloud
echo ""
cd build && docker build -t hpedevcloud . && cd ..
echo ""
docker run -d -p $HTTPPORT:80 -v `pwd`/data:/data --name hpedevcloud hpedevcloud:latest
docker image ls
docker container ls

Now reload the nextcloud configuration page in your browser, but this time configure the data folder as in the following screen shot:

![Nextcloud Setup](Pictures/nextcloud_setup.png)

** If you encounter issues, it's probably because you need to adapt the rights so that the apache user is allowed to write on to the /data directory. **

If you're stuck, just run the following command:

In [ ]:
ls -al data
id=`docker exec hpedevcloud grep apache /etc/passwd |cut -d: -f3`
echo "Apache user uid is $id and will own data"
docker exec hpedevcloud chown -R $id:$id data 
ls -al data

Obviously you can see the side effect of being root in the container, able to use chown and change owner of files on the host. Of course, you're limited to what you exposed in the container. Now this is solved, setup nextcloud again as proposed upper! Click on the files icon at the top and the plus icon to upload a PDF file into your nextcloud instance.

![Nextcloud Upload](Pictures/nextcloud_upload.png)

Check that you can open it from nextcloud.

Check that stoping and relaunching your nextcloud container now provides persistency to data:

In [ ]:
docker stop hpedevcloud
docker rm hpedevcloud
echo ""
cd build && docker build -t hpedevcloud . && cd ..
echo ""
docker run -d -p $HTTPPORT:80 -v `pwd`/data:/data --name hpedevcloud hpedevcloud:latest
docker image ls
docker container ls

And reload your browser page, log again and verify the file is still there. What ? You're back to setup again ? Well, that's not a surprise no ! I told you that it was based on trial and errors. In fact nextcloud uses a second directory to store config data, the config dir:

In [ ]:
docker exec hpedevcloud cat /var/www/html/nextcloud/config/config.php

So you see information around the storage of data and the admin user. So again we need to externalize this directory if we want persistence across restart. Let's do it, after cleanup so our nexcloud is fresh again:

In [ ]:
mkdir -p config
ls -al config
id=`docker exec hpedevcloud grep apache /etc/passwd |cut -d: -f3`
echo "Cleaning up docker images and containers"
docker container prune -f
docker image prune -f
docker stop hpedevcloud
docker rm hpedevcloud
docker rmi hpedevcloud:latest
docker image ls
docker container ls
echo ""
cd build && docker build -t hpedevcloud . && cd ..
echo ""
docker run -d -p $HTTPPORT:80 -v `pwd`/data:/var/www/html/nextcloud/data -v `pwd`/config:/var/www/html/nextcloud/config --name hpedevcloud hpedevcloud:latest
docker exec hpedevcloud touch /var/www/html/nextcloud/config/CAN_INSTALL
echo "Apache user uid is $id and will own config"
docker exec hpedevcloud chown -R $id:$id /var/www/html/nextcloud/config  /var/www/html/nextcloud/data
docker image ls
docker container ls

Now setup again your nextcloud, keeping all the defaults this time as we remapped the data directory directly in the default expected place. Upload again a file. Stop the container again and restart it and check that now you can directly login and that the file is still there.

In [ ]:
docker stop hpedevcloud
docker rm hpedevcloud
docker run -d -p $HTTPPORT:80 -v `pwd`/data:/var/www/html/nextcloud/data -v `pwd`/config:/var/www/html/nextcloud/config --name hpedevcloud hpedevcloud:latest
docker image ls
docker container ls

Great. So as you can see, it's not necessarily easy to deal with storage for acontainerized application, and we have not even touched the network ;-)
But with what we've done, we can alreay see that some help would be needed to manage command line parameters, and more over a case where we would want to add a real database instead of SQLite in a second container. Let's look at what docker-compose provides then.

<br><br>

## <i class="fas fa-2x fa-map-marker-alt" style="color:#551199;"></i>&nbsp;&nbsp;Next Steps

# Lab4 : Using Docker Compose

<h2>Next LAB&nbsp;&nbsp;&nbsp;&nbsp;<a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next LAB: Using Docker Compose"><i class="fas fa-chevron-circle-right" style="color:#551199;"></i></a></h2>

</br>
 <a href="1-WKSHP-Intro-to-Containers-techno.ipynb" target="New" title="Back: Introduction to Containers technologies"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: left;"><b>Back</b></button></a>
 <a href="4-WKSHP-Using-Docker-Compose.ipynb" target="New" title="Next:Using Docker Compose"><button type="submit"  class="btn btn-lg btn-block" style="background-color:#551199;color:#fff;position:relative;width:10%; height: 30px;float: right;"><b>Next</b></button></a>
